# The Extended Kalman Filter

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

### Robot Motion Model

At a first approximation an automobile steers by pivoting the front tires while moving forward. The front of the car moves in the direction that the wheels are pointing while pivoting around the rear tires. This simple description is complicated by issues such as slippage due to friction, the differing behavior of the rubber tires at different speeds, and the need for the outside tire to travel a different radius than the inner tire. Accurately modeling steering requires a complicated set of differential equations. 

For lower speed robotic applications a simpler *bicycle model* has been found to perform well. This is a depiction of the model:

In the **Unscented Kalman Filter** chapter we derived these equations:

$$\begin{aligned} 
\beta &= \frac d w \tan(\alpha) \\
x &= x - R\sin(\theta) + R\sin(\theta + \beta) \\
y &= y + R\cos(\theta) - R\cos(\theta + \beta) \\
\theta &= \theta + \beta
\end{aligned}
$$

where $\theta$ is the robot's heading.

You do not need to understand this model in detail if you are not interested in steering models. The important thing to recognize is that our motion model is nonlinear, and we will need to deal with that with our Kalman filter.

### Design the State Variables

For our filter we will maintain the position $x,y$ and orientation $\theta$ of the robot:

$$\mathbf x = \begin{bmatrix}x \\ y \\ \theta\end{bmatrix}$$

Our control input $\mathbf u$ is the velocity $v$ and steering angle $\alpha$:

$$\mathbf u = \begin{bmatrix}v \\ \alpha\end{bmatrix}$$

### Design the System Model

We model our system as a nonlinear motion model plus noise.

$$\bar x = f(x, u) + \mathcal{N}(0, Q)$$



Using the motion model for a robot that we created above, we can expand this to

$$\bar{\begin{bmatrix}x\\y\\\theta\end{bmatrix}} = \begin{bmatrix}x\\y\\\theta\end{bmatrix} + 
\begin{bmatrix}- R\sin(\theta) + R\sin(\theta + \beta) \\
R\cos(\theta) - R\cos(\theta + \beta) \\
\beta\end{bmatrix}$$

We find The $\mathbf F$ by taking the Jacobian of $f(x,u)$.

$$\mathbf F = \frac{\partial f(x, u)}{\partial x} =\begin{bmatrix}
\frac{\partial f_1}{\partial x} & 
\frac{\partial f_1}{\partial y} &
\frac{\partial f_1}{\partial \theta}\\
\frac{\partial f_2}{\partial x} & 
\frac{\partial f_2}{\partial y} &
\frac{\partial f_2}{\partial \theta} \\
\frac{\partial f_3}{\partial x} & 
\frac{\partial f_3}{\partial y} &
\frac{\partial f_3}{\partial \theta}
\end{bmatrix}
$$

When we calculate these we get

$$\mathbf F = \begin{bmatrix}
1 & 0 & -R\cos(\theta) + R\cos(\theta+\beta) \\
0 & 1 & -R\sin(\theta) + R\sin(\theta+\beta) \\
0 & 0 & 1
\end{bmatrix}$$

We can double check our work with SymPy.

In [ ]:
import sympy
from sympy.abc import alpha, x, y, v, w, R, theta
from sympy import symbols, Matrix

sympy.init_printing(use_latex="mathjax", fontsize="16pt")
time = symbols("t")
d = v * time
beta = (d / w) * sympy.tan(alpha)
r = w / sympy.tan(alpha)

fxu = Matrix(
    [
        [x - r * sympy.sin(theta) + r * sympy.sin(theta + beta)],
        [y + r * sympy.cos(theta) - r * sympy.cos(theta + beta)],
        [theta + beta],
    ]
)
F = fxu.jacobian(Matrix([x, y, theta]))
F

That looks a bit complicated. We can use SymPy to substitute terms:

In [ ]:
# reduce common expressions
B, R = symbols("beta, R")
F = F.subs((d / w) * sympy.tan(alpha), B)
F.subs(w / sympy.tan(alpha), R)

This form  verifies that the computation of the Jacobian is correct.

Now we can turn our attention to the noise. Here, the noise is in our control input, so it is in *control space*. In other words, we command a specific velocity and steering angle, but we need to convert that into errors in $x, y, \theta$. In a real system this might vary depending on velocity, so it will need to be recomputed for every prediction. I will choose this as the noise model; for a real robot you will need to choose a model that accurately depicts the error in your system. 

$$\mathbf{M} = \begin{bmatrix}\sigma_{vel}^2 & 0 \\ 0 & \sigma_\alpha^2\end{bmatrix}$$

If this was a linear problem we would convert from control space to state space using the by now familiar $\mathbf{FMF}^\mathsf T$ form. Since our motion model is nonlinear we do not try to find a closed form solution to this, but instead linearize it with a Jacobian which we will name $\mathbf{V}$. 

$$\mathbf{V} = \frac{\partial f(x, u)}{\partial u} \begin{bmatrix}
\frac{\partial f_1}{\partial v} & \frac{\partial f_1}{\partial \alpha} \\
\frac{\partial f_2}{\partial v} & \frac{\partial f_2}{\partial \alpha} \\
\frac{\partial f_3}{\partial v} & \frac{\partial f_3}{\partial \alpha}
\end{bmatrix}$$

These partial derivatives become very difficult to work with. Let's compute them with SymPy. 

In [ ]:
V = fxu.jacobian(Matrix([v, alpha]))
V = V.subs(sympy.tan(alpha) / w, 1 / R)
V = V.subs(time * v / R, B)
V = V.subs(time * v, "d")
V

This should give you an appreciation of how quickly the EKF become mathematically intractable. 

This gives us the final form of our prediction equations:

$$\begin{aligned}
\mathbf{\bar x} &= \mathbf x + 
\begin{bmatrix}- R\sin(\theta) + R\sin(\theta + \beta) \\
R\cos(\theta) - R\cos(\theta + \beta) \\
\beta\end{bmatrix}\\
\mathbf{\bar P} &=\mathbf{FPF}^{\mathsf T} + \mathbf{VMV}^{\mathsf T}
\end{aligned}$$

This form of linearization is not the only way to predict $\mathbf x$. For example, we could use a numerical integration technique such as *Runge Kutta* to compute the movement
of the robot. This will be required if the time step is relatively large. Things are not as cut and dried with the EKF as for the Kalman filter. For a real problem you have to carefully model your system with differential equations and then determine the most appropriate way to solve that system. The correct approach depends on the accuracy you require, how nonlinear the equations are, your processor budget, and numerical stability concerns.

### Design the Measurement Model

The robot's sensor provides a noisy bearing and range measurement to multiple known locations in the landscape. The measurement model must convert the state $\begin{bmatrix}x & y&\theta\end{bmatrix}^\mathsf T$ into a range and bearing to the landmark. If $\mathbf p$ 
is the position of a landmark, the range $r$ is

$$r = \sqrt{(p_x - x)^2 + (p_y - y)^2}$$

The sensor provides bearing relative to the orientation of the robot, so we must subtract the robot's orientation from the bearing to get the sensor reading, like so:

$$\phi = \arctan(\frac{p_y - y}{p_x - x}) - \theta$$


Thus our measurement model $h$ is


$$\begin{aligned}
\mathbf z& = h(\bar{\mathbf x}, \mathbf p) &+ \mathcal{N}(0, R)\\
&= \begin{bmatrix}
\sqrt{(p_x - x)^2 + (p_y - y)^2} \\
\arctan(\frac{p_y - y}{p_x - x}) - \theta 
\end{bmatrix} &+ \mathcal{N}(0, R)
\end{aligned}$$

This is clearly nonlinear, so we need linearize $h$  at $\mathbf x$ by taking its Jacobian. We compute that with SymPy below.

In [ ]:
px, py = symbols("p_x, p_y")
z = Matrix(
    [[sympy.sqrt((px - x) ** 2 + (py - y) ** 2)], [sympy.atan2(py - y, px - x) - theta]]
)
z.jacobian(Matrix([x, y, theta]))

Now we need to write that as a Python function. For example we might write:

In [ ]:
from math import sqrt


def H_of(x, landmark_pos):
    """ compute Jacobian of H matrix where h(x) computes 
    the range and bearing to a landmark for state x """

    px = landmark_pos[0]
    py = landmark_pos[1]
    hyp = (px - x[0, 0]) ** 2 + (py - x[1, 0]) ** 2
    dist = sqrt(hyp)

    H = array(
        [
            [-(px - x[0, 0]) / dist, -(py - x[1, 0]) / dist, 0],
            [(py - x[1, 0]) / hyp, -(px - x[0, 0]) / hyp, -1],
        ]
    )
    return H

We also need to define a function that converts the system state into a measurement.

In [ ]:
from math import atan2


def Hx(x, landmark_pos):
    """ takes a state variable and returns the measurement
    that would correspond to that state.
    """
    px = landmark_pos[0]
    py = landmark_pos[1]
    dist = sqrt((px - x[0, 0]) ** 2 + (py - x[1, 0]) ** 2)

    Hx = array([[dist], [atan2(py - x[1, 0], px - x[0, 0]) - x[2, 0]]])
    return Hx

### Design Measurement Noise

It is reasonable to assume that the noise of the range and bearing measurements are independent, hence

$$\mathbf R=\begin{bmatrix}\sigma_{range}^2 & 0 \\ 0 & \sigma_{bearing}^2\end{bmatrix}$$

### Implementation

We will use `FilterPy`'s `ExtendedKalmanFilter` class to implement the filter. Its `predict()` method uses the standard linear equations for the process model. Ours is nonlinear, so we will have to override `predict()` with our own implementation.  I'll want to also use this class to simulate the robot, so I'll add a method `move()` that computes the position of the robot which both `predict()` and my simulation can call.

The matrices for the prediction step are quite large. While writing this code I made several errors before I finally got it working. I only found my errors by using SymPy's `evalf` function. `evalf` evaluates a SymPy `Matrix` with specific values for the variables. I decided to demonstrate this technique to you, and used `evalf` in the Kalman filter code. You'll need to understand a couple of points.

First, `evalf` uses a dictionary to specify the values. For example, if your matrix contains an `x` and `y`, you can write

```python
    M.evalf(subs={x:3, y:17})
```
    
to evaluate the matrix for `x=3` and `y=17`. 

Second, `evalf` returns a `sympy.Matrix` object. Use `numpy.array(M).astype(float)` to convert it to a NumPy array. `numpy.array(M)` creates an array of  type `object`, which is not what you want.

Here is the code for the EKF:

In [ ]:
from filterpy.kalman import ExtendedKalmanFilter as EKF
from numpy import array, sqrt


class RobotEKF(EKF):
    def __init__(self, dt, wheelbase, std_vel, std_steer):
        EKF.__init__(self, 3, 2, 2)
        self.dt = dt
        self.wheelbase = wheelbase
        self.std_vel = std_vel
        self.std_steer = std_steer

        a, x, y, v, w, theta, time = symbols("a, x, y, v, w, theta, t")
        d = v * time
        beta = (d / w) * sympy.tan(a)
        r = w / sympy.tan(a)

        self.fxu = Matrix(
            [
                [x - r * sympy.sin(theta) + r * sympy.sin(theta + beta)],
                [y + r * sympy.cos(theta) - r * sympy.cos(theta + beta)],
                [theta + beta],
            ]
        )

        self.F_j = self.fxu.jacobian(Matrix([x, y, theta]))
        self.V_j = self.fxu.jacobian(Matrix([v, a]))

        # save dictionary and it's variables for later use
        self.subs = {x: 0, y: 0, v: 0, a: 0, time: dt, w: wheelbase, theta: 0}
        self.x_x, self.x_y, = x, y
        self.v, self.a, self.theta = v, a, theta

    def predict(self, u):
        self.x = self.move(self.x, u, self.dt)

        self.subs[self.theta] = self.x[2, 0]
        self.subs[self.v] = u[0]
        self.subs[self.a] = u[1]

        F = array(self.F_j.evalf(subs=self.subs)).astype(float)
        V = array(self.V_j.evalf(subs=self.subs)).astype(float)

        # covariance of motion noise in control space
        M = array([[self.std_vel * u[0] ** 2, 0], [0, self.std_steer ** 2]])

        self.P = F @ self.P @ F.T + V @ M @ V.T

    def move(self, x, u, dt):
        hdg = x[2, 0]
        vel = u[0]
        steering_angle = u[1]
        dist = vel * dt

        if abs(steering_angle) > 0.001:  # is robot turning?
            beta = (dist / self.wheelbase) * tan(steering_angle)
            r = self.wheelbase / tan(steering_angle)  # radius

            dx = np.array(
                [
                    [-r * sin(hdg) + r * sin(hdg + beta)],
                    [r * cos(hdg) - r * cos(hdg + beta)],
                    [beta],
                ]
            )
        else:  # moving in straight line
            dx = np.array([[dist * cos(hdg)], [dist * sin(hdg)], [0]])
        return x + dx

Now we have another issue to handle. The residual is notionally computed as $y = z - h(x)$ but this will not work because our measurement contains an angle in it. Suppose z has a bearing of $1^\circ$ and $h(x)$ has a bearing of $359^\circ$. Naively subtracting them would yield a angular difference of $-358^\circ$, whereas the correct value is $2^\circ$. We have to write code to correctly compute the bearing residual.

In [ ]:
def residual(a, b):
    """ compute residual (a-b) between measurements containing 
    [range, bearing]. Bearing is normalized to [-pi, pi)"""
    y = a - b
    y[1] = y[1] % (2 * np.pi)  # force in range [0, 2 pi)
    if y[1] > np.pi:  # move to [-pi, pi)
        y[1] -= 2 * np.pi
    return y

In [ ]:
from filterpy.stats import plot_covariance_ellipse
from math import sqrt, tan, cos, sin, atan2
import matplotlib.pyplot as plt

dt = 1.0


def z_landmark(lmark, sim_pos, std_rng, std_brg):
    x, y = sim_pos[0, 0], sim_pos[1, 0]
    d = np.sqrt((lmark[0] - x) ** 2 + (lmark[1] - y) ** 2)
    a = atan2(lmark[1] - y, lmark[0] - x) - sim_pos[2, 0]
    z = np.array([[d + np.random.randn() * std_rng], [a + np.random.randn() * std_brg]])
    return z


def ekf_update(ekf, z, landmark):
    ekf.update(
        z, HJacobian=H_of, Hx=Hx, residual=residual, args=(landmark), hx_args=(landmark)
    )


def run_localization(
    landmarks,
    std_vel,
    std_steer,
    std_range,
    std_bearing,
    step=10,
    ellipse_step=20,
    ylim=None,
):
    ekf = RobotEKF(dt, wheelbase=0.5, std_vel=std_vel, std_steer=std_steer)
    ekf.x = array([[2, 6, 0.3]]).T  # x, y, steer angle
    ekf.P = np.diag([0.1, 0.1, 0.1])
    ekf.R = np.diag([std_range ** 2, std_bearing ** 2])

    sim_pos = ekf.x.copy()  # simulated position
    # steering command (vel, steering angle radians)
    u = array([1.1, 0.01])

    plt.figure()
    plt.scatter(landmarks[:, 0], landmarks[:, 1], marker="s", s=60)

    track = []
    for i in range(200):
        sim_pos = ekf.move(sim_pos, u, dt / 10.0)  # simulate robot
        track.append(sim_pos)

        if i % step == 0:
            ekf.predict(u=u)

            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.x[0, 0], ekf.x[1, 0]),
                    ekf.P[0:2, 0:2],
                    std=6,
                    facecolor="k",
                    alpha=0.3,
                )

            x, y = sim_pos[0, 0], sim_pos[1, 0]
            for lmark in landmarks:
                z = z_landmark(lmark, sim_pos, std_range, std_bearing)
                ekf_update(ekf, z, lmark)

            if i % ellipse_step == 0:
                plot_covariance_ellipse(
                    (ekf.x[0, 0], ekf.x[1, 0]),
                    ekf.P[0:2, 0:2],
                    std=6,
                    facecolor="g",
                    alpha=0.8,
                )
    track = np.array(track)
    plt.plot(track[:, 0], track[:, 1], color="k", lw=2)
    plt.axis("equal")
    plt.title("EKF Robot localization")
    if ylim is not None:
        plt.ylim(*ylim)
    plt.show()
    return ekf

In [ ]:
landmarks = array([[5, 10], [10, 5], [15, 15]])

ekf = run_localization(
    landmarks, std_vel=0.1, std_steer=np.radians(1), std_range=0.3, std_bearing=0.1
)
print("Final P:", ekf.P.diagonal())

I have plotted the landmarks as solid squares. The path of the robot is drawn with a black line. The covariance ellipses for the predict step are light gray, and the covariances of the update are shown in green. To make them visible at this scale I have set the ellipse boundary at 6$\sigma$.

We can see that there is a lot of uncertainty added by our motion model, and that most of the error in in the direction of motion. We determine that from the shape of the blue ellipses. After a few steps we can see that the filter incorporates the landmark measurements and the errors improve.

I used the same initial conditions and landmark locations in the UKF chapter. The UKF achieves much better accuracy in terms of the error ellipse. Both perform roughly as well as far as their estimate for $\mathbf x$ is concerned. 

Now let's add another landmark.

In [ ]:
landmarks = array([[5, 10], [10, 5], [15, 15], [20, 5]])

ekf = run_localization(
    landmarks, std_vel=0.1, std_steer=np.radians(1), std_range=0.3, std_bearing=0.1
)
plt.show()
print("Final P:", ekf.P.diagonal())

The uncertainly in the estimates near the end of the track are smaller.  We can see the effect that multiple landmarks have on our uncertainty by only using the first two landmarks.

In [ ]:
ekf = run_localization(
    landmarks[0:2], std_vel=1.0e-10, std_steer=1.0e-10, std_range=1.4, std_bearing=0.05
)
print("Final P:", ekf.P.diagonal())

The estimate quickly diverges from the robot's path after passing the landmarks. The covariance also grows quickly. Let's see what happens with only one landmark:

In [ ]:
ekf = run_localization(
    landmarks[0:1], std_vel=1.0e-10, std_steer=1.0e-10, std_range=1.4, std_bearing=0.05
)
print("Final P:", ekf.P.diagonal())

As you probably suspected, one landmark produces a very bad result. Conversely, a large number of landmarks allows us to make very accurate estimates.

In [ ]:
landmarks = array(
    [
        [5, 10],
        [10, 5],
        [15, 15],
        [20, 5],
        [15, 10],
        [10, 14],
        [23, 14],
        [25, 20],
        [10, 20],
    ]
)

ekf = run_localization(
    landmarks,
    std_vel=0.1,
    std_steer=np.radians(1),
    std_range=0.3,
    std_bearing=0.1,
    ylim=(0, 21),
)
print("Final P:", ekf.P.diagonal())